In [15]:
import numpy as np

In [19]:
import pickle

In [20]:
training_data_11k = []
with open('training_data_50k.pkl' ,'rb')as f :
    training_data_11k = pickle.load(f)

In [21]:
# training_data_11k[0]

In [22]:
pip install -U gensim

Requirement already up-to-date: gensim in c:\users\asus\anaconda3\lib\site-packages (4.1.2)
Note: you may need to restart the kernel to use updated packages.


In [23]:
import multiprocessing
from gensim.models import Word2Vec
import time
import numpy as np
from numpy.linalg import norm

In [24]:
cores = multiprocessing.cpu_count()
print('Number of cores in this device {}'.format(cores))

Number of cores in this device 8


In [25]:
docs_num = len(training_data_11k)
tokens_num = sum([len(x) for x in training_data_11k])
print('Number of all docs {}'.format(docs_num))
print('Number of all Tokens {}'.format(tokens_num))

Number of all docs 50061
Number of all Tokens 12325127


In [26]:
%%time
w2v_model = Word2Vec(min_count=1 , window=15 , vector_size=300 , alpha=0.03 , workers=cores-1)
w2v_model.build_vocab(training_data_11k)
w2v_model_vocab_size = len(w2v_model.wv)
print('Vocab size : ' , w2v_model_vocab_size)

Vocab size :  140103
Wall time: 5.22 s


In [27]:
# start = time.time()
# w2v_model.train(training_data_11k , total_examples=w2v_model.corpus_count , epochs= 30)
# end = time.time()
# print("{} s". format(end-start))

In [28]:
w2v_model = Word2Vec.load("w2v_50k_300d.model")

In [29]:
docs_tfidf_11k = []
with open('tfidf_50k.pkl' ,'rb')as f :
    docs_tfidf_11k = pickle.load(f)

In [30]:
# docs_tfidf_11k[11000]

In [31]:
def make_docs_embeding(docs_tfidf):
    docs_embedding = []
    for doc in docs_tfidf:
        doc_vector = np.zeros(300)
        weight_sum = 0
        if(len(doc.items())!=0):
            
       
            for token , weight in doc.items() :


                        doc_vector += w2v_model.wv[token] * weight
                        weight_sum += weight

            docs_embedding.append(doc_vector/weight_sum)
        
        
    
 
        
    return docs_embedding

In [32]:
docs_embedding_11k = make_docs_embeding(docs_tfidf_11k)
with open('docs_embedding_50k.pkl' , 'wb') as f :
        pickle.dump(docs_embedding_11k,f)

In [33]:
def similarity(doc1 , doc2) :
    similarity_score = np.dot(doc1 , doc2) / ((norm(doc1)) *(norm(doc2)) )
    return (similarity_score +1)/2

In [34]:
s = similarity(docs_embedding_11k[11] , docs_embedding_11k[10])
s

0.8867190140366563

K-Means :)

In [35]:
import random

In [36]:
# Random Seed
def random_seed(docs_embedding):
    seeds = []
    for i in range(5):
        x = random.randint(0,len(docs_embedding))
        seeds.append(docs_embedding[x])
    return seeds

In [37]:
type(docs_embedding_11k) 

list

In [38]:
def update_seeds_and_reassign(seeds , docs_embedding):
    
    clusters = [[] , [] ,[] ,[] ,[]]
    for index ,doc in enumerate(docs_embedding) :

            best_similarity = 0
            cluster = 6 # 6 means no cluster
            flag = False
            for s in range(len(seeds)) :
                if(not np.array_equal(doc,seeds[s])):
                    sim = similarity(doc , seeds[s])

                    if np.float64(sim).item() > best_similarity :
                        best_similarity = sim
                        cluster = s
                        flag = True
                        
                    
                       
                       
            
            if(not np.array_equal(doc,seeds[s])):       
                clusters[cluster].append((doc , index))
    
    #update seeds
    new_seeds = []
    for i in range(len(clusters)):
        sum1 = np.zeros(300)
        for c in clusters[i] :
            sum1 += c[0]
        new_seeds.append(sum1/(len(clusters[i])))
   
    return new_seeds , clusters
            
    

In [39]:
# As we consider similarity the max the better
def calculate_error(clusters , seeds , docs_embedding) :
    Rss = 0 
    for i in range(len(clusters)) :    
        for c in clusters[i]:
            Rss += similarity(c[0], seeds[i])
    return Rss
            

In [40]:
def k_means(docs_embedding):
    best_clustering = []
    max_error = 0

    # random seed
    seeds = random_seed(docs_embedding)

    for i in range(10) :
        seeds , clusters = update_seeds_and_reassign(seeds , docs_embedding)
        error = calculate_error(clusters , seeds , docs_embedding)
        if(i != 0) :
            if(error > max_error):
                max_error = error
                best_clustering = clusters
        else :
            max_error = error
            best_clustering = clusters
    
    return best_clustering , seeds , max_error
       

In [41]:
best_clustering , seeds , min_error = k_means(docs_embedding_11k)
clustering_11k = {'clusters' :best_clustering , 'seeds' : seeds}
with open('clustering_50k.pkl' , 'wb') as f :
        pickle.dump(clustering_11k,f)

In [51]:
import hazm
hazm_normalaizer = hazm.Normalizer()
lemmatizer = hazm.Lemmatizer()

In [43]:
from collections import Counter
import math
def query_embeding(query ,docs_num) :
       
        normal_query = hazm_normalaizer.normalize(query)
        tokens = hazm.word_tokenize(normal_query)
        tokens_stems = []
        for t in tokens :
            stem_token = lemmatizer.lemmatize(t)
            if("#" in stem_token):
                mazi , mozare = stem_token.split("#")
                tokens_stems.append(mazi)
                tokens_stems.append(mozare)
            else :
                tokens_stems.append(stem_token)
            
        token_tfidf = {}

        
        for term, count in Counter(tokens_stems).items():
            tf = (1 + math.log10(count))
            idf = math.log10(docs_num/1)
            tfidf = tf * idf
            token_tfidf [term] = tfidf
        
        query_vector = np.zeros(300)
        weight_sum = 0
        for token , weight in token_tfidf.items() :
            try:
            
                query_vector += w2v_model.wv[token] * weight
                
            except KeyError:
                continue
            weight_sum += weight
        if weight_sum == 0 :
            return query_vector
 
     
        return query_vector/weight_sum

In [44]:
import xlrd
data_set = '50k_combined.xlsm'
data_reader = xlrd.open_workbook(data_set)
content = data_reader.sheet_by_index(0)

In [45]:
def answer_query(query) :
    answers ={}
    
    
    with open('docs_embedding_50k.pkl' ,'rb')as f :
        docs_embedding_11k = pickle.load(f)
        f.close()

    with open('clustering_50k.pkl' ,'rb')as f :
        clustering_11k = pickle.load(f)
        f.close()

    seeds = clustering_11k['seeds']
    clusters = clustering_11k['clusters']
        
        
    query_embed = query_embeding(query , 50061)
    best_cluster = 0
    best_similarity = 0
    for i in range(len(seeds)):
        sim = similarity(query_embed , seeds[i])
        if(np.float64(sim).item() > best_similarity) :
            best_similarity = sim
            best_cluster = i
    
    for c in clusters[best_cluster] :
        sim = similarity(query_embed , c[0])
        if(sim > 0.70) :
            answers [c[1]] = sim
   
    answers = dict(sorted(answers.items(), key=lambda item: item[1], reverse=True))
    
    for d in answers:
        print("{} : {}".format(d,answers[d]))
        title = data_reader.sheet_by_index(0).cell(int(d), 2).value
        url =  data_reader.sheet_by_index(0).cell(int(d), 3).value
        print(title)
        print(url)
        print("================================================")
    

In [50]:
query = input("Enter your query")
answer_query(query)

Enter your queryآمار کرونا
5499 : 0.7682768484032887
health
https://www.isna.ir/news/99012514449/شناسایی-۱۶۱۷-بیمار-جدید-کرونا-در-کشور-111-تن-جان-باختند
5434 : 0.7670907220460018
health
https://www.isna.ir/news/99010703174/تضعیف-حس-بویایی-و-چشایی-از-شکایت-های-اصلی-بیماران-مشکوک-به-کرونا
6163 : 0.7661835534079995
health
https://www.isna.ir/news/99081409123/شناسایی-۸۴۵۲-مبتلای-جدید-کرونا-و-فوت-۴۱۹-بیمار-دیگر
23082 : 0.7659224209058124
economy
https://www.farsnews.ir/news/13990307000915/عرضه-0-هزار-میلیارد-تومان-اوراق-بدهی-دولت-در-هفته-آینده-نرخ-سود-اوراق
23638 : 0.765097509878294
economy
https://www.farsnews.ir/news/13990224000423/تجهیز-۲۴-میلیون-هکتار-از-اراضی-کشاورزی-به-سیستم‌های-نوین-آبیاری
12991 : 0.764301628653254
economy
https://www.farsnews.ir/news/13991210000741/کاهش-قابل-توجه-تولید-نفت-آمریکا
18152 : 0.763210090214429
economy
https://www.farsnews.ir/news/13990716000132/هنوز-یک-سوم-ارز-مصوب-برای-واردات-کالاهای-اساسی-را-دریافت-نکرده-ایم
31263 : 0.761793984054562
sports
https://www